In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install diffusers transformers accelerate scipy safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 16.2 MB/s eta 0:00:00


In [4]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.26.1
aiohttp                          3.9.1
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.11.2
b

In [5]:
!pip install diffusers

In [16]:
import torch
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image
import os
import glob
import time
import sys

# this is a function to test if the image is completely black
# It is used in order to circumvent the problem of images being
# flagged as NSFW
def is_image_completely_black(image):
    try:
        width, height = image.size
        for x in range(width):
            for y in range(height):
                pixel = image.getpixel((x, y))
                if pixel != (0, 0, 0):
                    return False
        return True
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

#Load the stable diffusion inpainting pipeline
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "your path here",
)

pathToImages = "your path here"
pathToMasks = "your path here"
pathToOutput = "your path here"

pipe.to("cuda")
pipe.enable_attention_slicing()

counter = 0
for imagename in os.listdir(pathToImages):
  try:
    print(imagename)
    imagePath = os.path.join(pathToImages, imagename)
    image_in = Image.open(imagePath)
    image_path_without_png = imagename.replace(".png", pathToImages)
    image_name_without_png = imagename.replace(".png", "")
    mask_image_path = pathToMasks + "/" + image_name_without_png + "_mask.png"
    mask_image = Image.open(mask_image_path)
    prompt = "an sks black background with large white rectangles"
    seed=0
    generator = torch.Generator("cuda").manual_seed(seed)
    image = pipe(prompt=prompt, image=image_in, mask_image=mask_image, generator=generator).images[0]
    #image = pipe(prompt=prompt, image=image_in, mask_image=mask_image).images[0] # use this line instead of the above in case you don't want to use a specific seed
    outputname = pathToOutput + imagename + "_inpainting_result_" + str(counter) + ".png"
    image.save(outputname)

    # Optional, try again with another seed while the image output is flagged as NSFW
    # Trying with a different seed in case the image is flagged as NSFW
    trycounter = 0
    max_attempts = 20

    while is_image_completely_black(image) and trycounter < max_attempts:
        # Adjust the seed
        seed = 2 ** (trycounter + 2)
        generator = torch.Generator("cuda").manual_seed(seed)

        # Perform inpainting
        image = pipe(prompt=prompt, image=image_in, mask_image=mask_image, generator=generator).images[0]

        # Save the output image
        output_name = f"{pathToOutput}{imagename}_inpainting_result_{counter}.png"
        image.save(output_name)

        print(f"Trycounter: {trycounter}")
        trycounter += 1

    if is_image_completely_black(image):
        print("Failed to generate a non-black image after multiple attempts.")
    else:
        print("Inpainting successfully completed.")
  except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {str(e)}")
  counter = counter + 1
print("Alle erledigt")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

conflict_map_8.png


  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 